In [1]:
import yfinance as yf
import pandas as pd
from datetime import date
import numpy as np

In [2]:
nifty = yf.download('^NSEI',start="2010-01-01",end=date.today())
nifty.reset_index(inplace=True)
nifty = nifty.droplevel(level=1,axis=1)
nifty.tail(5)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^NSEI']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Date,Adj Close,Close,High,Low,Open,Volume


In [3]:
nifty['SMA_10']=nifty['Close'].rolling(window=10).mean()
nifty['SMA_50']=nifty['Close'].rolling(window=50).mean()
nifty['EMA_20']=nifty['Close'].ewm(span=20).mean()
nifty['price_change']=nifty['Close'].pct_change(periods=1)
nifty['volatility']= nifty['Close'].rolling(window=10).std()

In [4]:
nifty['target'] = nifty['Close'].shift(-1)

In [5]:
nifty.dropna(inplace=True)
nifty = nifty.drop(columns='Date',axis=1)

In [6]:
nifty.tail()

Price,Adj Close,Close,High,Low,Open,Volume,SMA_10,SMA_50,EMA_20,price_change,volatility,target


In [7]:
from sklearn.preprocessing import MinMaxScaler
Min_max = MinMaxScaler()
nifty_norm = Min_max.fit_transform(nifty)

ValueError: Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
x_data = nifty.drop(columns='target',axis=1)
y_data = nifty['target']
def create_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X.iloc[i:i+window_size].values)
        ys.append(y.iloc[i+window_size])
    return np.array(Xs), np.array(ys)

window_size = 10
X_seq, y_seq = create_sequences(x_data, y_data, window_size)

split_index = int(len(X_seq) * 0.8)
x_train, x_test = X_seq[:split_index], X_seq[split_index:]
y_train, y_test = y_seq[:split_index], y_seq[split_index:]


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM

2025-06-03 21:45:52.705882: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 21:45:55.789502: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 21:45:57.307963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748967358.665283    3945 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748967358.912666    3945 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748967361.633681    3945 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
reg = Sequential()
reg.add(LSTM(units=50, activation="relu", input_shape=(window_size, x_data.shape[1])))
reg.add(Dense(units=1))
reg.compile(optimizer="adam", loss="mean_squared_error")
reg.fit(x_train, y_train, batch_size=32, epochs=100)


/home/mg/Downloads/python/jupyter/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 483606976.0000
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 219216256.0000
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 99578536.0000
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 112818424.0000
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 34393176.0000
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 44492500.0000
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 28931326.0000
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 131366832.0000
Epoch 9/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 44861716.0000
Epoch 10/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 41223092.0000
Epoch 11/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 10414062.0000
Epoch 12/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 22239924.0000
Epoch 13/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 9894045.0000
Epoch 14/100
94/94 ━━━━━━━━━━━━━━

In [ ]:
yhat = reg.predict(x_test)
from sklearn.metrics import r2_score,mean_squared_error,root_mean_squared_error
print(r2_score(y_test, yhat), mean_squared_error(y_test, yhat),root_mean_squared_error(y_test,yhat))


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
0.7998864067773815 1657614.1097332616 1287.4836347438602
